In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import cv2

from PIL import Image
from math import ceil
from pynq import Overlay, allocate

%matplotlib inline 

XSA = "Interupt_Red_Filter.xsa"
overlay = Overlay(XSA)
help(overlay)

Help on Overlay in module pynq.overlay:

<pynq.overlay.Overlay object>
    Default documentation for overlay Interupt_Red_Filter.xsa. The following
    attributes are available on this overlay:
    
    IP Blocks
    ----------
    axi_intc_0           : pynq.overlay.DefaultIP
    vdma                 : pynq.lib.video.dma.AxiVDMA
    v_tpg_0              : pynq.overlay.DefaultIP
    processing_system7_0 : pynq.overlay.DefaultIP
    
    Hierarchies
    -----------
    RGB_Filter_0         : pynq.overlay.DefaultHierarchy
    
    Interrupts
    ----------
    None
    
    GPIO Outputs
    ------------
    None
    
    Memories
    ------------
    PSDDR                : Memory



In [2]:
vdma = overlay.vdma

In [3]:
width, height = 1280, 720
input_buffer = allocate(shape=(height,width,3), dtype=np.uint8, cacheable=1)
adress_of_input_buffer = input_buffer.device_address

vdma.write(0x00, 0x04)
while vdma.read(0x00)& 0x4 == 4:
    pass

vdma.write(0x30, 0x04)
while vdma.read(0x30)& 0x4 == 4:
    pass

# Config
vdma.write(0x30, 0x8003)

# Frame buffers
vdma.write(0xAC, adress_of_input_buffer)
vdma.write(0xB0, 0x102A4000)
vdma.write(0xB4, 0x10548000)

# HSIZE
vdma.write(0xA8, width * 3)
vdma.write(0xA4, width * 3)

# VSIZE
vdma.write(0xA0, height)

# Read
vdma.write(0x00, 0x8B)
vdma.write(0x5C, adress_of_input_buffer)
vdma.write(0x60, 0x102A4000)
vdma.write(0x64, 0x10548000)
vdma.write(0x58, width * 3)
vdma.write(0x54, width * 3)
vdma.write(0x50, height)

In [ ]:
print("starting stream")
RTSP_URL = "https://commondatastorage.googleapis.com/gtv-videos-bucket/sample/BigBuckBunny.mp4"
cap = cv2.VideoCapture(RTSP_URL)

if not cap.isOpened():
    print("Error: Unable to open the RTSP stream.")
    exit()

print("RTSP stream opened successfully.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Could not retrieve frame.")
        break
    resized_frame = cv2.resize(frame, (width, height))
    input_buffer[:] = np.array(resized_frame)

starting stream
RTSP stream opened successfully.
